<a href="https://colab.research.google.com/github/bcorr24/Final-Project-Data-301/blob/master/Data_Cleaning_and_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Collection

In [0]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import time

In [0]:
#import the mvp url and pass through beautiful soup
import requests
response = requests.get("https://www.basketball-reference.com/awards/mvp.html")
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")
tables = soup.find_all("table")

In [3]:
#run through tables and web scrape the mvp table
rows = []
for i in [0,1]:
  for faculty in tables[i].find_all("tr")[2:]:
    # Get all the cells in the row.
    stats = faculty.find_all("td")
    seasons = faculty.find_all("a")

    #get each variable
    name = seasons[2].text
    season = seasons[0].text
    team = seasons[4].text

    age = stats[3].text
    games = stats[5].text
    MP = stats[6].text
    PTS= stats[7].text	
    TRB= stats[8].text	
    AST= stats[9].text
    STL=stats[10].text	
    BLK=stats[11].text	
    FG=stats[12].text	
    three= stats[13].text	
    FT=stats[14].text	
    WS=stats[15].text	
    WSpergame= stats[16].text
    
    # Append this data.
    rows.append({
        "Season":season,
        "Player":name,
        "Team":team,
        "Age":age,	
        "games_played":games,	
        "mpg":MP	,
        "ppg":PTS,
        "rpg":TRB,
        "apg":AST,
        "spg":STL,
        "bpg":BLK,
        "FG%":FG,
        "3%":three,
        "FT%":FT,
        "WS":WS,
        "WSpg":WSpergame
    })
#put into a dataframe
df_mvp = pd.DataFrame(rows)
df_mvp.head(10)

,Season,Player,Team,Age,games_played,mpg,ppg,rpg,apg,spg,bpg,FG%,3%,FT%,WS,WSpg
0,2018-19,Giannis Antetokounmpo,MIL,24,72,32.8,27.7,12.5,5.9,1.3,1.5,.578,.256,.729,14.4,.292
1,2017-18,James Harden,HOU,28,72,35.4,30.4,5.4,8.8,1.8,0.7,.449,.367,.858,15.4,.289
2,2016-17,Russell Westbrook,OKC,28,81,34.6,31.6,10.7,10.4,1.6,0.4,.425,.343,.845,13.1,.224
3,2015-16,Stephen Curry,GSW,27,79,34.2,30.1,5.4,6.7,2.1,0.2,.504,.454,.908,17.9,.318
4,2014-15,Stephen Curry,GSW,26,80,32.7,23.8,4.3,7.7,2.0,0.2,.487,.443,.914,15.7,.288
5,2013-14,Kevin Durant,OKC,25,81,38.5,32.0,7.4,5.5,1.3,0.7,.503,.391,.873,19.2,.295
6,2012-13,LeBron James,MIA,28,76,37.9,26.8,8.0,7.3,1.7,0.9,.565,.406,.753,19.3,.322
7,2011-12,LeBron James,MIA,27,62,37.5,27.1,7.9,6.2,1.9,0.8,.531,.362,.771,14.5,.298
8,2010-11,Derrick Rose,CHI,22,81,37.4,25.0,4.1,7.7,1.0,0.6,.445,.332,.858,13.1,.208
9,2009-10,LeBron James,CLE,25,76,39.0,29.7,7.3,8.6,1.6,1.0,.503,.333,.767,18.5,.299


In [0]:
#web scraping set up for allstars
url = "https://www.basketball-reference.com/awards/all_star_by_player.html"
import requests
response = requests.get(url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")
tables = soup.find_all("table")

In [0]:
#get the url from each all star webpage in order to scrap for their stats
names=[]
new= []
for faculty in tables[0].find_all("tr")[1:]:
    # Get all the cells in the row.
    row = faculty.find_all("td")

    name = row[1].text
    url =row[1].find_all('a', href=True)[0]['href']


    # Append this data.
    names.append(name)
    new.append(url)
urls= []
for i in new:
  url = "https://www.basketball-reference.com" + i
  urls.append(url)


In [0]:
#for each url scrape players for stats

#p.s. takes almost 3 min to run

#intialize df
all_stars = pd.DataFrame()

#iterate through urls
for i in range(len(urls)):
  #set up web scraping
  response = requests.get(urls[i])
  soup = BeautifulSoup(response.content, "html.parser")
  tables = soup.find_all("table")
  new= tables[0].find_all("tr")[1:]
  rows=[]
  for faculty in new:
    #look only at those who were mvp's (had a star by their name)
    if faculty.find("span"):
      stats = faculty.find_all("td")
      seasons = faculty.find_all("a")

      #some of the tables were set up in different ways so i had to make clauses
      if len(faculty)<28:

        season = seasons[0].text
        team = seasons[1].text

        age = stats[0].text
        position = stats[3].text
        games = stats[4].text
        MP = stats[6].text
        PTS= stats[-1].text	
        TRB= stats[15].text	
        AST= stats[16].text
        if len(stats)>18:         
          STL=stats[17].text	
          BLK=stats[18].text	
        else:
          STL=''
          BLK=''
        if len(stats)<=17:
          TRB= stats[14].text	
          AST= stats[15].text


        FG=stats[9].text	
        three= ""	
        FT=stats[12].text	

      else:
        season = seasons[0].text
        team = seasons[1].text

        age = stats[0].text
        position = stats[3].text
        games = stats[4].text
        MP = stats[6].text
        PTS= stats[-1].text	
        TRB= stats[22].text	
        AST= stats[23].text
        STL=stats[24].text	
        BLK=stats[25].text	
        FG=stats[9].text	
        three= stats[12].text	
        FT=stats[19].text	
    
      # After getting all the correct variables, append to rows.
      rows.append({
        "Season":season,
        "Player":names[i],
        "Position":position,
        "Team":team,
        "Age":age,	
        "games_played":games,	
        "mpg":MP	,
        "ppg":PTS,
        "rpg":TRB,
        "apg":AST,
        "spg":STL,
        "bpg":BLK,
        "FG%":FG,
        "3%":three,
        "FT%":FT
      })
      #make a data frame of the new all star and append it to the all star df
      all_stars = pd.concat([all_stars, pd.DataFrame(rows)], ignore_index=True)
      


In [7]:
all_stars.head(10)

,Season,Player,Position,Team,Age,games_played,mpg,ppg,rpg,apg,spg,bpg,FG%,3%,FT%
0,1969-70,Kareem Abdul-Jabbar,C,MIL,22,82,43.1,28.8,14.5,4.1,,,.518,,.653
1,1969-70,Kareem Abdul-Jabbar,C,MIL,22,82,43.1,28.8,14.5,4.1,,,.518,,.653
2,1970-71,Kareem Abdul-Jabbar,C,MIL,23,82,40.1,31.7,16.0,3.3,,,.577,,.690
3,1969-70,Kareem Abdul-Jabbar,C,MIL,22,82,43.1,28.8,14.5,4.1,,,.518,,.653
4,1970-71,Kareem Abdul-Jabbar,C,MIL,23,82,40.1,31.7,16.0,3.3,,,.577,,.690
5,1971-72,Kareem Abdul-Jabbar,C,MIL,24,81,44.2,34.8,16.6,4.6,,,.574,,.689
6,1969-70,Kareem Abdul-Jabbar,C,MIL,22,82,43.1,28.8,14.5,4.1,,,.518,,.653
7,1970-71,Kareem Abdul-Jabbar,C,MIL,23,82,40.1,31.7,16.0,3.3,,,.577,,.690
8,1971-72,Kareem Abdul-Jabbar,C,MIL,24,81,44.2,34.8,16.6,4.6,,,.574,,.689
9,1972-73,Kareem Abdul-Jabbar,C,MIL,25,76,42.8,30.2,16.1,5.0,,,.554,,.713


##Data Cleaning

In [0]:
#some had duplicates
all_stars = all_stars.drop_duplicates(keep="first")

In [0]:
#Some of the players had played for 2 teams in one season so i wanted to 
#combine them into one season with averages

#first I created a df with only those who had 2 teams in one season
diff_teams = all_stars[all_stars.duplicated(subset=["Season", "Player"],
                                            keep=False)]
diff_teams = diff_teams[diff_teams["Team"] != ("NBA")]
diff_teams = diff_teams[diff_teams["Team"] != ("ABA")]
diff_teams = diff_teams.reset_index()
#make quantitative variables
diff_teams[["games_played", "mpg",	"ppg",	"rpg",	"apg",	"spg",	"bpg",	
            "FG%",	"3%",	"FT%"]] = diff_teams[["games_played", "mpg",
                        "ppg",	"rpg",	"apg",	"spg",
                        "bpg",	"FG%",	"3%",	"FT%"]].apply(pd.to_numeric, 
                                                      errors='coerce', axis=1)

#iterate through and get averages of the two by games played and make variables
L=[]
for i in range(0,len(diff_teams),2):
  Player = diff_teams.iloc[i]['Player']
  Season = diff_teams.iloc[i]['Season']
  Position = diff_teams.iloc[i]['Position']
  team = diff_teams.iloc[i]['Team'] + "/" + diff_teams.iloc[i+1]['Team']
  age = diff_teams.iloc[i]['Age']

  g1= diff_teams.iloc[i]['games_played']
  g2= diff_teams.iloc[i+1]['games_played']


  games = diff_teams.iloc[i]['games_played'] + diff_teams.iloc[i+1
                                                               ]['games_played']
  MP = (g1 * diff_teams.iloc[i]['mpg'] + g2*diff_teams.iloc[i]['mpg'])/games
  PTS = (g1 *diff_teams.iloc[i]['ppg'] + g2*diff_teams.iloc[i]['ppg'])/games
  TRB = (g1 *diff_teams.iloc[i]['rpg'] + g2*diff_teams.iloc[i]['rpg'])/games
  AST = (g1 *diff_teams.iloc[i]['apg'] + g2*diff_teams.iloc[i]['apg'])/games
  STL = (g1 *diff_teams.iloc[i]['spg'] + g2*diff_teams.iloc[i]['spg'])/games
  BLK = (g1 *diff_teams.iloc[i]['bpg'] + g2*diff_teams.iloc[i]['bpg'])/games
  FG = (g1 *diff_teams.iloc[i]['FG%'] + g2*diff_teams.iloc[i]['FG%'])/games
  three = (g1 *diff_teams.iloc[i]['3%'] + g2*diff_teams.iloc[i]['3%'])/games
  FT = (g1 *diff_teams.iloc[i]['FT%'] + g2*diff_teams.iloc[i]['FT%'])/games


  diff_teams.iloc[i+1]['Player']

  L.append({
        "Season":season,
        "Player":Player,
        "Position":Position,
        "Team":team,
        "Age":age,	
        "games_played":games,	
        "mpg":MP	,
        "ppg":PTS,
        "rpg":TRB,
        "apg":AST,
        "spg":STL,
        "bpg":BLK,
        "FG%":FG,
        "3%":three,
        "FT%":FT

  })

#make it a data frame
duplicates =pd.DataFrame(L)

In [0]:
#drop all the duplicates and concat the new df to the old all stars
all_stars = all_stars.drop_duplicates(subset=["Season", "Player"],keep=False)
all_stars = pd.concat([all_stars, duplicates], ignore_index=True, sort = False)

In [0]:
#adding mvp dummy variable to playes who won using my mvp df
mvp_seasons = []
for i in range(len(df_mvp)):
  play = df_mvp.iloc[i]["Player"]
  seas = df_mvp.iloc[i]["Season"]
  mvp_seasons.append((play,seas))

mvps_list = [] 
for i in range(len(all_stars)):
  play = all_stars.iloc[i]["Player"]
  seas = all_stars.iloc[i]["Season"]
  if (play, seas) in mvp_seasons:
    mvps_list.append(1)
  else:
    mvps_list.append(0)

#make it a new column
all_stars["MVP"] = mvps_list

In [13]:
from google.colab import drive
drive.mount('/content/gdrive')
all_stars.to_csv('full_data.csv', index=False)
!cp full_data.csv "/content/gdrive/My Drive/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
from google.colab import drive
full_data = pd.read_csv('/content/gdrive/My Drive/full_data.csv')
full_data[full_data["Position"].isna()]

,Season,Player,Position,Team,Age,games_played,mpg,ppg,rpg,apg,spg,bpg,FG%,3%,FT%,MVP
328,1950-51,Dolph Schayes,NaN,SYR,22,66.0,NaN,17.0,3.8,4.1,NaN,NaN,0.357,NaN,0.752,0
713,1950-51,Harry Gallatin,NaN,NYK,23,66.0,NaN,12.8,2.7,3.7,NaN,NaN,0.416,NaN,0.732,0
748,1950-51,Ed Macauley,NaN,BOS,22,68.0,NaN,20.4,3.7,3.0,NaN,NaN,0.466,NaN,0.759,0
1149,1950-51,Andy Phillip,NaN,PHW,28,66.0,NaN,11.2,6.3,3.3,NaN,NaN,0.399,NaN,0.751,0
1151,1952-53,Andy Phillip,NaN,NBA,30,70.0,38.4,10.3,5.7,3.3,NaN,NaN,0.397,NaN,0.738,0
1534,1950-51,Frankie Brian,NaN,TRI,27,68.0,NaN,16.8,3.9,3.2,NaN,NaN,0.322,NaN,0.823,0
1640,1954-55,Frank Selvy,NaN,NBA,22,71.0,37.6,19.0,3.5,3.2,NaN,NaN,0.378,NaN,0.728,0
1698,1950-51,Ralph Beard,NaN,INO,23,66.0,NaN,16.8,4.8,1.5,NaN,NaN,0.368,NaN,0.775,0
1741,1950-51,Alex Groza,NaN,INO,24,66.0,NaN,21.7,2.4,3.6,NaN,NaN,0.470,NaN,0.786,0
1817,1950-51,Fred Schaus,NaN,FTW,25,68.0,NaN,15.1,2.7,3.5,NaN,NaN,0.340,NaN,0.835,0
